## Air Quality in Vanderbijlpark Prediction (LINEAR REGRESSION)

In [ ]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [ ]:

def wrangle(filename):
    #We read the filename into a dataframe and passing the ';' as a delimiter
    df = pd.read_csv(filename,delimiter=';')
    # We convert our timestamp column of object datatype to a timestamp datatype
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    # We set the index of the dataframe to be the timestamp 
    df.set_index('timestamp',inplace=True)
    #We  convert the timestamp to the 'Africa/Johannesburg' timezone since the dataset is gotten from Vanderbijlpark
    df.index = df.index.tz_convert('Africa/Johannesburg')
    #Sort Our data Frame by index
    df = df.sort_index()
    #We mask our DataFrame to return value that contains the P2 value type
    mask = df['value_type'] == 'P2'
    df  = df[mask]
    ## we identify column with low and high cardinality and drop them
    low_cardinality_list = [column for column in df.columns if df[column].nunique() < 2 ]
    df.drop(low_cardinality_list,axis=1,inplace = True)
    # We resample the dataframe to return a series with the mean of the P2 value for every Hour
    y =  df['value'].resample('1H').mean().fillna(method ='ffill')
    y.name = 'P2'
    return y
    

### 1. IMPORT

In [ ]:
#Returns all files that matches the pattern as specified in the glob function argument
files = glob('data/*_2021_sensor_data_archive.csv')

In [ ]:
#A list comprehension for that perform the wrangle function for each file in the list created by glob funtion
list_of_df = [wrangle(file) for file in files]
# We concatenate the list of dataframes into one dataframe using the pd.concat method and set index to the timestamp
y = pd.concat(list_of_df).sort_index()

### 2. EXPLORE

In [ ]:
#We plot the weekly rolling average to get an idea of how the data could change in future
fig,ax = plt.subplots(figsize=(25,7))
y.rolling(168).mean().plot(ax=ax);
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.title('Rolling Average of PM2.5 With Time');


In [ ]:
#Since we have no feature and only a target value, we could create a lag feature from the target.
#First we convert our series to a DataFrame
df = y.to_frame()
df.head()

In [ ]:
# Next we create a new feature called 'P2.Lag1' which is the value from previous row
df['P2.Lag1'] = df['P2'].shift(1)
df.dropna(inplace=True)

In [ ]:
df.info()

In [ ]:
#Next we create a heatmap between the feature and target
fig,ax = plt.subplots(figsize=(15,6))
sns.heatmap(df.corr(),annot=True,ax=ax,cmap='Blues')
#Title
plt.title('A scatter plot of `P2` against `P2.Lag`');

In [ ]:
fig,ax = plt.subplots(figsize=(15,6))
#We plot a scatter plot of 'P2' against 'P2.Lag'
plt.scatter(data = df,x='P2.Lag1',y='P2')
#We plot a straight line from the origin to check our distributed our scatter plot.
plt.plot([0,400],[0,400],linestyle='solid',color='red')
#xlabel
plt.xlabel('P2.Lag1')
plt.ylabel('P2')
plt.title('A scatter plot of `P2` against `P2.Lag`');

### 3. SPLIT

In [ ]:
# Split DataFrame in X(Feature) and y(Target).
target = 'P2'
# Feature column
X = df.drop(target,axis=1)
y = df[target]

In [ ]:
#We split into Train and Test set. Since it is a Time Based Data, We ensure no future data is present in the test set.
# We Split in the Ratio 80:20
cut_off = int(len(X)*0.8)
X_train = X[:cut_off]
X_test = X[cut_off:]
y_train = y[:cut_off]
y_test = y[cut_off:]

### 4. BUILD MODEL

#### A. BASELINE

In [ ]:
# We create a baseline model which would be used to test how well our test data those.
# The baseline will be the mean absolute error of y_train against that of y_mean
y_mean = y_train.mean()
y_pred_baseline = [y_mean]*len(y_train)
y_baseline_df = pd.DataFrame({'y_train' : y_train,'y_baseline':y_pred_baseline})
mae = mean_absolute_error(y_train,y_pred_baseline)
print(f' The mean of the y_train is {y_mean}')
print(f' The mean absolute error of the baseline model is {mae}')

In [ ]:
fig = px.line(y_baseline_df,labels={'value':'P2'},title='A Time Series Plot of the y_train and y_baseline',width=1300)
fig.show('png');

### B. LINEAR REGRESSION MODEL

In [ ]:
#We instantiate the model and fit to the training data
lr_model = LinearRegression() 
lr_model.fit(X_train,y_train)

In [ ]:
#Next we predict the X_train and calculate the mean absolute error
y_train_pred = lr_model.predict(X_train)
lr_mae = mean_absolute_error(y_train,y_train_pred)
print(f' The mean absolute error of the Linear Regression model with the train test is {lr_mae}')

In [ ]:
# Next we make prediction of X_test and compare with y_test
y_test_pred = lr_model.predict(X_test)
lr_test_mae = mean_absolute_error(y_test,y_test_pred)
print(f' The mean absolute error of the Linear Regression model with the test set is {lr_test_mae}')

### 5. COMMUNICATE RESULT

In [ ]:
# We extract the coefficent and intercept
intercept = round(lr_model.intercept_,2)
coef = round(lr_model.coef_[0],2)
print(f' The intercept of the linear regression model is {intercept} and the coefficient is {coef}, therefore equation is:')
print(f'\n\n\t\t\t P2 = (P2.Lag1) x {coef} +  {intercept}')

In [ ]:
y_pred_df = pd.DataFrame({
                'y_test':y_test,
                'y_prediction':y_test_pred
})
y_pred_df.head()

In [ ]:
y_pred_df.info()

In [ ]:
fig = px.line(y_pred_df,labels={'value':'P2'},title='A Time Series Plot of the y_test and y_pred',width=1500)
fig.show('png');